# MCMC fitting
In this notebook, I will read in the RV data and perform the MCMC fit to find the mass-ratio.

[for now, I am writing up how to do an SB2 fit. If that fails again, I will fall back to a partial fit using Stefano's parameters.]

In [2]:
import numpy as np
import pandas as pd
import MassRatio_Fitter
import triangle
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
# Read in the austral rv fits
rv1_data = pd.read_fwf('data/psi1draa_140p_28_37_ASW.dat', header=None)
t1 = rv1_data[0].values
rv1 = rv1_data[2].values / 1000. # Convert from m/s to km/s
rv1_err = rv1_data[3].values / 1000.

# Read in my measurements for the companion RV
t2, _, _, rv2, rv2_err = np.loadtxt('data/rv_data.txt')
good = ~np.isnan(rv2)
t2 = t2[good]
rv2 = rv2[good]
rv2_err = rv2_err[good]

In [ ]:
# Do the fit
sampler = MassRatio_Fitter.full_sb2_fit(t1, rv1, rv1_err, t2, rv2, rv2_err)